In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.linear_model import LogisticRegression

In [31]:
#1. Pima Indian Diabetes 데이터 세트 diabetes.csv 파일을 불러오고 데이터의 평균, 표준편차, 4분위수를 체크하세요. 
#데이터프레임의 첫 다섯 행도 확인하세요.

diabetes_df = pd.read_csv('./diabetes.csv')
diabetes_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [32]:
diabetes_df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [33]:
diabetes_df['Pregnancies'].value_counts()

1     135
0     111
2     103
3      75
4      68
5      57
6      50
7      45
8      38
9      28
10     24
11     11
13     10
12      9
14      2
15      1
17      1
Name: Pregnancies, dtype: int64

In [34]:
#2. 'Glucose', 'BloodPressure','SkinThickness','Insulin','BMI' 변수에서 0인 값들을 평균 값으로 대체하세요.
zero_features = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
total_count = diabetes_df['Glucose'].count()

for feature in zero_features:
    zero_count = diabetes_df[diabetes_df[feature]==0][feature].count()
    print('{0} 0 건수는 {1}, 전체의 {2:.2f}%입니다'.format(feature, zero_count, 100*zero_count/total_count))

Glucose 0 건수는 5, 전체의 0.65%입니다
BloodPressure 0 건수는 35, 전체의 4.56%입니다
SkinThickness 0 건수는 227, 전체의 29.56%입니다
Insulin 0 건수는 374, 전체의 48.70%입니다
BMI 0 건수는 11, 전체의 1.43%입니다


In [35]:
mean_zero_features = diabetes_df[zero_features].mean()
diabetes_df[zero_features] = diabetes_df[zero_features].replace(0, mean_zero_features)

In [28]:
#3. Pregnancies 별로 당뇨병에 걸릴 확률을 나타낸 그래프를 그리세요.



In [30]:
#4. 데이터셋을 피처 데이터셋, 결과 데이터셋으로 나눠주세요.

x = diabetes_df.iloc[:,:-1]
y = diabetes_df.iloc[:,-1]

In [ ]:
#5. MinMaxScaler 을 이용해 피처 데이트셋에 스케일링을 적용하고, #train/test 데이터셋을 train_test_split을 이용하여7:3으로 나눠주세요.

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=156, stratify=y)

In [37]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [45]:
#6. Logistic Regression을 이용하여 학습, 예측 한 뒤 accuracy_score 을 구하세요.

lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)
pred = lr_clf.predict(x_test)
pred_proba = lr_clf.predict_proba(x_test)[:,1]

get_clf_eval(y_test, pred, pred_proba)

오차 행렬
[[90 10]
 [21 33]]
정확도: 0.7987, 정밀도: 0.7674, 재현율: 0.6111,    F1: 0.6804, AUC:0.8443


In [46]:
#7. 해당 모델로 교차검증을 하고 평균 정확도를 구하세요.

scores = cross_val_score(lr_clf, x, y, cv=5)
for iter_count, accuracy in enumerate(scores):
    print("교차검증 {0} 정확도 {1:.4f}".format(iter_count, accuracy))

print("평균 정확도: {0:.4f}".format(np.mean(scores)))

교차검증 0 정확도 0.7468
교차검증 1 정확도 0.7792
교차검증 2 정확도 0.7468
교차검증 3 정확도 0.8235
교차검증 4 정확도 0.7516
평균 정확도: 0.7696


In [48]:
def get_eval_by_threshold(y_test , pred_proba_c1, thresholds):
    # thresholds 리스트 객체내의 값을 차례로 iteration하면서 Evaluation 수행.
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값:',custom_threshold)
        get_clf_eval(y_test , custom_predict, pred_proba_c1)

In [51]:
#8. 임곗값(threshold)을 0.52로 설정한 binarizer 을 생성하세요. 
#그리고 위에서 구한 logistic regression 모델의predict_proba() 예측 확률 array에서 positive(1)에 해당하는 컬럼을 binarize 하세요.

thresholds = [0.52]
pred_proba = lr_clf.predict_proba(x_test)

binarizer = Binarizer(threshold=0.52)

pred_th_052 = binarizer.fit_transform(pred_proba[:,1].reshape(-1,1))

In [52]:
#9. 8번에서 구한 예측 결과를 토대로 confusion matrix, accuracy, precision, recall, f1, roc 값을 구하세요.

get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1, 1), thresholds)

임곗값: 0.52
오차 행렬
[[90 10]
 [21 33]]
정확도: 0.7987, 정밀도: 0.7674, 재현율: 0.6111,    F1: 0.6804, AUC:0.8443
